In [16]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deploy ADK Agent in Agent Engine

This notebook provides a step-by-step to deploy an Agent Created using Agent Development Kit on Agent Engine (ReasonEngine on Vertex AI)

**Important**: This notebook consider that the Agent was built with ADK and the agent files are inside an agent folder and the dependencies in a file config.yaml

Folders structure (example): 
```
parent_folder/
    agent_folder/
        __init__.py
        agent.py
        config.yaml
        ... # other files
        .env
    deploy_agent_engine.ipynb
```



### Setup and Config

In [17]:
# Checking the google-adk and google-cloud-aiplatform versions
!pip freeze | grep google-adk
!pip freeze | grep google-cloud-aiplatform

google-adk==1.10.0
google-cloud-aiplatform==1.108.0


In [18]:
# Authentication on gcloud (if necessary)
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=eQBgXo353vyUBaftRUfH4abz5Z823C&access_type=offline&code_challenge=YWH-i0zEXM2Nvxk7r8NOovdBFNoOp4dN89l5rFNUznI&code_challenge_method=S256


Credentials saved to file: [/Users/junioroliveira1662/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "i8-n8n-prod" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [3]:
# Basic Libraries
import os 
import vertexai
import yaml

# AI Engine on Vertex AI 
from vertexai import agent_engines

# Library for AI Engine with ADK
from vertexai.preview import reasoning_engines

# Just to view JSON response formatted
import json
from IPython.display import display,Markdown,JSON

# To load envvars dict from .env file
from dotenv import dotenv_values

/Users/junioroliveira1662/Projects/gitlab/estudos-ai/agent-kanbanize/.venv/lib/python3.13/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "config_type" in "SequentialAgent" shadows an attribute in parent "BaseAgent"
  warnings.warn(


In [4]:
# Load Agent Config
AGENT_DIR = "kanbanize_agent"

# Load environment variables from .env file from agent Directory 
from dotenv import load_dotenv
env_file = f'./{AGENT_DIR}/.env'
load_dotenv(env_file)

# Load config from agents (Params and dependencies for deploy)
with open(f'./{AGENT_DIR}/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# For Vertex AI SDK 
PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")
LOCATION = os.environ.get("GOOGLE_CLOUD_LOCATION")
BUCKET = os.environ.get("GOOGLE_CLOUD_BUCKET")

print(f"PROJECT_ID = {PROJECT_ID}")
print(f"LOCATION = {LOCATION}")
print(f"BUCKET = {BUCKET}")


PROJECT_ID = i8-n8n-prod
LOCATION = us-central1
BUCKET = gs://vertexai-deploy


### Instantiate Agent from Directory

In [5]:
# Importing Agent Module from AGENT_DIR folder
import importlib
agent_module = importlib.import_module(f"{AGENT_DIR.replace('/','.')}.agent")

# Instantiate the Assistant as an ADK App 
adk_agent = reasoning_engines.AdkApp(
    agent=agent_module.root_agent,
    enable_tracing=True
)

### Running Agent Local (Optional)

In [6]:
# Run a simple query
for event in adk_agent.stream_query(
    user_id="user",
    message="Hi, how can you help me?",
):
    pass

# Formatted output
display(Markdown(f"```json\n{json.dumps(event, indent=2,ensure_ascii=False)}\n```"))

```json
{
  "content": {
    "parts": [
      {
        "thought_signature": "Cu0BAcu98PCLQZIDpyZc4o4mWueZ5vrOTPjrxNMZkMlYIcs9VnXOyNpg4NHbgUBY4V1RRq0STTg1y-WUisfWWu_le2_PHjJxxiJ7KXBwUgO8slultKvFS8g0yHlrXtGHG5vQWtlMpQc7Snct3yK7wRQYogYhj2NJcU-rGG5W9fsW-d6k-n2NnUnJknhmQb8XOheNtLbh-8-E1SB9o6qFKTDs_niHQ7mpHGKOAayf0X-EW2cfGAY-DcUQk2FAlVTbyNM9P-532TJZRRHKk194OhmQYrzh9ctI_UgDzjybcAaZAIJoJFaNqBdECP6XFZf3",
        "text": "Olá! Eu sou um especialista em gestão de backlog e posso te ajudar a criar e detalhar histórias (user stories) de forma eficaz para o Kanbanize.\n\nPosso te auxiliar com as seguintes tarefas:\n\n1.  **Entender o contexto da sua solicitação:** Vou fazer perguntas para garantir que a história atenda aos seus objetivos e à sua equipe.\n2.  **Formular user stories:** Ajudarei a escrever suas histórias no formato padrão: \"Como [tipo de usuário], quero [objetivo] para [benefício esperado]\".\n3.  **Definir critérios de aceite:** Vamos criar juntos critérios claros e verificáveis para que a equipe saiba exatamente o que precisa ser entregue.\n4.  **Criar descrições detalhadas:** Sugerirei informações importantes para incluir na descrição do card, facilitando a execução e o entendimento.\n5.  **Gerenciar cards no Kanbanize:**\n    *   Posso **criar novos cards** no Kanbanize para você, preenchendo todos os campos necessários como título, descrição, responsável, coluna, etc.\n    *   Posso **consultar cards existentes** em um board específico do Kanbanize.\n\nEm resumo, meu objetivo é garantir que suas histórias sejam bem estruturadas, detalhadas e prontas para serem trabalhadas pela sua equipe no Kanbanize!\n\nComo posso começar a te ajudar hoje?"
      }
    ],
    "role": "model"
  },
  "usage_metadata": {
    "candidates_token_count": 294,
    "candidates_tokens_details": [
      {
        "modality": "TEXT",
        "token_count": 294
      }
    ],
    "prompt_token_count": 3502,
    "prompt_tokens_details": [
      {
        "modality": "TEXT",
        "token_count": 3502
      }
    ],
    "thoughts_token_count": 43,
    "total_token_count": 3839,
    "traffic_type": "ON_DEMAND"
  },
  "invocation_id": "e-dada421f-47da-44e7-a447-30ed15fd4766",
  "author": "kanbanize_agent",
  "actions": {
    "state_delta": {},
    "artifact_delta": {},
    "requested_auth_configs": {}
  },
  "id": "e97c9d29-e966-47d2-b31b-edd5bbd64760",
  "timestamp": 1755009966.911472
}
```

### Deploy on Agent Engine

In [7]:
# Instantiate Vertex AI
vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=BUCKET,
)

In [8]:
## Retrieve all existent Agent Engine on your project
for agent in agent_engines.list():
    print(f"============================ \nAgent: {agent.display_name}\nResoruce Name: {agent.resource_name}\nCreated/updated at: {agent.update_time} \n\n" )

Agent: Jira Agent
Resoruce Name: projects/101060224713/locations/us-central1/reasoningEngines/5396640563636207616
Created/updated at: 2025-08-06 11:23:14.807609+00:00 




In [9]:
# Read Requirements for Agent from config file
# Usually this ['google-cloud-aiplatform[agent_engines]', 'google-adk', 'cloudpickle'] plus the packages that agent needs
requirements = config['deploy']['dependencies']
requirements

['google-cloud-aiplatform[agent_engines,adk]',
 'google-adk',
 'cloudpickle',
 'pydantic',
 'python-dotenv',
 'requests',
 'pyyaml']

In [10]:
# Extra packages from agent folder (This is all .py files inside Agent Directory)
extra_packages = [AGENT_DIR]
extra_packages

['kanbanize_agent']

In [11]:
# Load Variables on env_vars dict to be used when creating the Agent
env_vars = dotenv_values(dotenv_path=env_file)

# Remove GCP variables (this variables already are defined in Agent Engine and are reserved)
keys_to_remove = [
    "GOOGLE_GENAI_USE_VERTEXAI",
    "GOOGLE_CLOUD_PROJECT",
    "GOOGLE_CLOUD_LOCATION",
    "GOOGLE_CLOUD_BUCKET"
]

for key in keys_to_remove:
    env_vars.pop(key, None)

In [12]:
# Deploy the Agent on AI Engine (This takes a few minutes)
remote_agent = agent_engines.create(
    agent_engine = adk_agent,             # The Agent instantiated as ADK agent
    requirements=requirements,            # Requirements file
    extra_packages=extra_packages,        # Extra packages
    display_name=config['agent_display_name'],    # Display name  
    description=config['agent_description'],     # Description
    env_vars=env_vars                     # Env Vars dict
)

Identified the following requirements: {'google-cloud-aiplatform': '1.108.0', 'pydantic': '2.11.7', 'cloudpickle': '3.1.1'}
The final list of requirements: ['google-cloud-aiplatform[agent_engines,adk]', 'google-adk', 'cloudpickle', 'pydantic', 'python-dotenv', 'requests', 'pyyaml']
Using bucket vertexai-deploy
Wrote to gs://vertexai-deploy/agent_engine/agent_engine.pkl
Writing to gs://vertexai-deploy/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://vertexai-deploy/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/101060224713/locations/us-central1/reasoningEngines/8348679750633390080/operations/3639878285350928384
View progress and logs at https://console.cloud.google.com/logs/query?project=i8-n8n-prod
AgentEngine created. Resource name: projects/101060224713/locations/us-central1/reasoningEngines/8348679750633390080
To use this AgentEngine in another session:
agent_engine = vertexai.agent_engines.ge

### Test Remote Agent on Agent Engine

In [15]:
## Retrieve all existent Agent Engine resource.names (Agents)
# To confirm new agent was deployed
for agent in agent_engines.list():
    print(f"============================ \nAgent: {agent.display_name}\nResoruce Name: {agent.resource_name}\nCreated/updated at: {agent.update_time} \n\n" )

Agent: Kanbanize Agent
Resoruce Name: projects/101060224713/locations/us-central1/reasoningEngines/8348679750633390080
Created/updated at: 2025-08-12 14:50:06.583398+00:00 


Agent: Jira Agent
Resoruce Name: projects/101060224713/locations/us-central1/reasoningEngines/5396640563636207616
Created/updated at: 2025-08-06 11:23:14.807609+00:00 




In [14]:
# Confirm that "remote_agent" is pointing to your new agent
print(f"=================== Remote Agent ============================ \n\
 Name: {remote_agent.display_name}\n\
 Resoruce Name: {remote_agent.resource_name}\n\
 Created/updated at: {remote_agent.update_time} \n\n" )

=================== Remote Agent ============================ 
 Name: Kanbanize Agent
 Resoruce Name: projects/101060224713/locations/us-central1/reasoningEngines/8348679750633390080
 Created/updated at: 2025-08-12 14:50:06.583398+00:00 




In [16]:
# Run a simple query
for remote_event in remote_agent.stream_query(
    user_id="user",
    message="Hi, how can you help me?",
):
    display(JSON(remote_event,expanded=True)) 

<IPython.core.display.JSON object>

In [17]:
# Formatted final output
display(Markdown(f"```json\n{json.dumps(remote_event, indent=2,ensure_ascii=False)}\n```"))

```json
{
  "content": {
    "parts": [
      {
        "text": "Hello! I can help you by searching for information within Kanbanize. You can ask me questions about tasks, projects, and their statuses, and I'll find the relevant details for you. For example, you can ask me to:\n\n*   \"Show me all tasks in the 'POC ADK GOOGLE' project.\"\n*   \"Find all issues assigned to me that are not yet resolved.\"\n*   \"List all high-priority stories.\"\n\nJust tell me what you'd like to know!"
      }
    ],
    "role": "model"
  },
  "usage_metadata": {
    "candidates_token_count": 104,
    "candidates_tokens_details": [
      {
        "modality": "TEXT",
        "token_count": 104
      }
    ],
    "prompt_token_count": 2915,
    "prompt_tokens_details": [
      {
        "modality": "TEXT",
        "token_count": 2915
      }
    ],
    "thoughts_token_count": 67,
    "total_token_count": 3086,
    "traffic_type": "ON_DEMAND"
  },
  "invocation_id": "e-e49cfe61-72b3-421b-9ac4-37f7790e4137",
  "author": "kanbanize_agent",
  "actions": {
    "state_delta": {},
    "artifact_delta": {},
    "requested_auth_configs": {}
  },
  "id": "953cb592-9e05-4d40-b291-17b976f9ed18",
  "timestamp": 1754482000.251909
}
```